In [10]:
import time
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine, Float
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table
import os

Base = declarative_base()
DBSession = scoped_session(sessionmaker())
#engine = None
print("ekhane asi1")

def init_sqlalchemy(dbname='sqlite:///chicago.eimu.db'):
    global engine
    engine = create_engine(dbname, echo=False)
    DBSession.remove()
    DBSession.configure(bind=engine, autoflush=False, expire_on_commit=False)
    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)
    print("ekhane asi2")

def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    print("ekhane asi3")
    return data

def create_table_from_csv(file_path):
    data = load_data_from_csv(file_path)
    metadata = Base.metadata
    print('hello')
    table_name = "CrimeEimuTable"  # Specify the table name
    CrimeEimuTable = Table(
        table_name,
        metadata,
        Column('IDbySQL', Integer, primary_key=True)
    )

    for column_name, dtype in data.dtypes.items():
        if dtype == 'int64':
            column = Column(column_name, Integer)
        elif dtype == 'float64':
            column = Column(column_name, Float)
        else:
            column = Column(column_name, String)
        CrimeEimuTable.append_column(column)

    metadata.create_all(engine)
    print("ekhane asi4")
    return CrimeEimuTable

def bulk_insert_data(table, data):
    DBSession.bulk_insert_mappings(table, data)
    DBSession.commit()
    print("ekhane asi5")


ekhane asi1


C:\Users\Resam Zaha\AppData\Local\Temp\ipykernel_13024\858761720.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [11]:

init_sqlalchemy()

# Load data from CSV (replace 'your_data.csv' with your dataset file)
csv_file_path = 'C://Study//CSE 4000//New folder//Thesis//dataset//New folder//20weather_data_sf_2004_2023.csv'
print("ekhane asi6")
# Split the CSV into 5 parts
chunk_size = pd.read_csv(csv_file_path, chunksize=10)  # You can adjust the chunk size as needed


ekhane asi2
ekhane asi6


In [12]:
#print(len(chunk_size))
table = create_table_from_csv('C://Study//CSE 4000//New folder//Thesis//dataset//New folder//20weather_data_sf_2004_2023.csv')

# Insert data chunk by chunk
for i, chunk in enumerate(chunk_size):
    print(f"Processing Chunk {i + 1} ...")
    
    class EimuModel(Base):
        __table__ = table
    bulk_insert_data(EimuModel, chunk.to_dict(orient='records'))
    print(f"Chunk {i + 1} processed.")
    print("ekhane asi7")

print("All data inserted into the database.")

TypeError: object of type 'TextFileReader' has no len()